### **Fine Tuning a model using the Trainer class of PyTorch.**

**Preprocessing**

In [12]:
# required initial processing for fine-tuning
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

# call dataset. Call tokenizer from a checkpoint
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Tokenize the dataset and batch it. Truncating it is more efficient than padding to the max, batching makes it faster for training
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# Pad the dataset to the max of the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
from transformers import TrainingArguments, AutoModelForSequenceClassification

# Define a TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evaluation. 
# "test-trainer" is the directory we will be storing the trained model and the checkpoints.
training_args = TrainingArguments("test-trainer")

# Define model with two labels
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Training with a compute metric for each epoch**

In [14]:
import evaluate

# define a compute metric for evaluation of the model after each epoch:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import Trainer

# Define a Trainer class with this compute_metrics() function:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# This will report the validation loss and metrics at the end of each epoch on top of the training loss
trainer.train()

c:\Users\antoi\miniconda3\envs\nlp\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.46050047874450684, 'eval_accuracy': 0.7843137254901961, 'eval_f1': 0.8294573643410853, 'eval_runtime': 10.2144, 'eval_samples_per_second': 39.944, 'eval_steps_per_second': 4.993, 'epoch': 1.0}
{'loss': 0.5705, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.5407851934432983, 'eval_accuracy': 0.8333333333333334, 'eval_f1': 0.8896103896103896, 'eval_runtime': 11.281, 'eval_samples_per_second': 36.167, 'eval_steps_per_second': 4.521, 'epoch': 2.0}
{'loss': 0.3763, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.5439541935920715, 'eval_accuracy': 0.8627450980392157, 'eval_f1': 0.902439024390244, 'eval_runtime': 10.8515, 'eval_samples_per_second': 37.599, 'eval_steps_per_second': 4.7, 'epoch': 3.0}
{'train_runtime': 412.1965, 'train_samples_per_second': 26.696, 'train_steps_per_second': 3.341, 'train_loss': 0.4122557851310735, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.4122557851310735, metrics={'train_runtime': 412.1965, 'train_samples_per_second': 26.696, 'train_steps_per_second': 3.341, 'train_loss': 0.4122557851310735, 'epoch': 3.0})

**Evaluation**

In [25]:
# outputs three fields (named tuples): predictions, label_ids, and metrics
# metrics field will just contain the loss on the dataset passed, as well as some time metrics (how long it took to predict, in total and on average).
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

preds = np.argmax(predictions.predictions, axis=-1)
print(preds)

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

  0%|          | 0/51 [00:00<?, ?it/s]

(408, 2) (408,)
[1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 0 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1
 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1
 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1
 0 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 0
 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0
 1]


{'accuracy': 0.8431372549019608, 'f1': 0.891156462585034}